# Prepare MNIST Dataset
This notebook describes how you can manually prepare the MNIST dataset for use with the rest of the sample code in Chapter 4 of the book. 

In this notebook we'll download the MNIST training and validation files, unpack them and then convert them to a file format supported by the CNTK minibatch source. 

The output of this notebook is a set of two files:

 * `mnist_train.ctf` The CNTK file containing the training samples
 * `mnist_test.ctf` The CNTK file containing the test samples

## Utility functions
This section of the notebook contains the utility functions for the data processing.
We'll use a function to load the image data and another separate function to load the labels.
Please check out the contents of the functions for more information on how they work.

In [1]:
import os
import gzip
import struct
import numpy as np
from urllib.request import urlretrieve

In [2]:
def load_data(url, image_count):
    """
    Downloads a data file from the specified URL and decodes it as a image_countx28x28 numpy array.
    
    The function expects that you specify an URL that hosts a file with the expected number of images.
    We'll do a check in the function if you specified the right file URL. This function will raise
    an error otherwise.
    
    Also, the images in the source file should be 28x28 pixels large. Any other format is not supported
    by this function and raises an error.
    
    Parameters
    ----------
    url : str
        The source URL for the data file
    image_count : int
        The number of images expected in the input file
    
    Returns
    -------
    A numpy array containing the images from the input file.
    """
    print('Downloading ', url)
    
    filename, h = urlretrieve(url, './.datafile')
    
    try:
        with gzip.open(filename) as gz_file:
            # Read the header of the file. This should contain a magic string that identifies the file.
            n = struct.unpack('>I', gz_file.read(4))
            
            if n[0] != 0x00000803:
                raise Warning('Invalid file. Wrong magic number encountered {}'.format(n))
            
            # Read the number of images stored in the file. This should match the expected image count.
            # We'll raise an error when it doesn't match, so be careful which file you download!
            n = struct.unpack('>I', gz_file.read(4))[0]
            
            if n != image_count:
                raise Exception('Invalid number of entries in file.')
                
            # Load the number of rows and columns from the file. This tells us how big the images really are.
            # They should both be 28, our model will not be able to process anything else.
            row_count = struct.unpack('>I', gz_file.read(4))[0]
            col_count = struct.unpack('>I', gz_file.read(4))[0]
            
            if row_count != 28 or col_count != 28:
                raise Exception("Invalid image size detected in file. {}, {}".format(row_count, col_count))
            
            # Reconstruct the original numpy array from the input file.
            # Please be aware, large files probably won't fit in memory so this will fail 
            # on a machine with low memory availability.
            result = np.frombuffer(gz_file.read(image_count * row_count * col_count), dtype=np.uint8)
            
    finally:
        # Make sure we remove the temporary file.
        # No need to keep it around.
        os.remove(filename)
    
    return result.reshape(image_count, row_count * col_count)

In [3]:
def load_labels(url, image_count):
    """
    Downloads a labels file from the specified URL and decodes as a image_countx1 numpy array.
    
    The function expects that you specify an URL that hosts a file with the expected number of images.
    We'll do a check in the function if you specified the right file URL. This function will raise
    an error otherwise.
    
    Parameters
    ----------
    url : str
        The URL for the labels file
    image_count : int
        The number of expected images in the file
        
    Returns
    -------
    A numpy array with the labels.
    """
    print('Downloading ', url)
    
    filename, h = urlretrieve(url, './.labelsfile')
    
    try:
        with gzip.open(filename) as gz_file:
            # Read the header of the file. This should contain a magic string that identifies the file.
            n = struct.unpack('I', gz_file.read(4))
            
            if n[0] != 0x1080000:
                raise Exception('File format eror. Unexpected header.')
            
            # Read the number of images stored in the file. This should match the expected image count.
            # We'll raise an error when it doesn't match, so be careful which file you download!
            n = struct.unpack('>I', gz_file.read(4))[0]
            
            if n != image_count:
                raise Exception('Invalid number of entries in file.')
                
            result = np.frombuffer(gz_file.read(image_count), dtype=np.uint8)  
    finally:
        # Make sure we remove the temporary file.
        # No need to keep it around.
        os.remove(filename)
        
    return result.reshape((image_count, 1))

In [4]:
def try_download(data_url, labels_url, image_count):
    data = load_data(data_url, image_count)
    labels = load_labels(labels_url, image_count)
    
    return data, labels

In [5]:
def save_data(filename, images, labels):
    output_directory = os.path.dirname(filename)
    
    # Make sure that the output folder exists.
    # This will save a lot of people from a major headache.
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    with open(filename, 'w') as output_file:
        for index in range(0, images.shape[0]):
            print('Processing image {} of {}'.format(index+1, images.shape[0]))
            image = images[index]
            label = labels[index][0]

            encoded_label = np.zeros((1, 10))
            encoded_label[0, label] = 1
            
            label_str = ' '.join(encoded_label[0].astype('str'))
            feature_str = ' '.join(image.astype('str'))
            
            output_file.write('|labels {} |features {}'.format(label_str, feature_str))

## Downloading the images
The first step in the preprocessing phase is to download the images and labels for the dataset.
We'll be loading two separate sets of data. One for training and one for validation.

In [6]:
train_labels_url = 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
train_images_url = 'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
train_samples = 60000

train_images, train_labels = try_download(train_images_url, train_labels_url, train_samples)

In [7]:
test_labels_url = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
test_images_url = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
test_samples = 10000

test_images, test_labels = try_download(test_images_url, test_labels_url, test_samples)

## Storing the data in a compatible format
We're going to need to store the data in a format that is compatible with the CNTK minibatch source.
A single call to `save_data` is all that is needed to create the right files. We'll use both files later to train and test the neural network.

In [8]:
save_data('./mnist_train.ctf', train_images, train_labels)

Processing image 1 of 60000
Processing image 2 of 60000
Processing image 3 of 60000
Processing image 4 of 60000
Processing image 5 of 60000
Processing image 6 of 60000
Processing image 7 of 60000
Processing image 8 of 60000
Processing image 9 of 60000
Processing image 10 of 60000
Processing image 11 of 60000
Processing image 12 of 60000
Processing image 13 of 60000
Processing image 14 of 60000
Processing image 15 of 60000
Processing image 16 of 60000
Processing image 17 of 60000
Processing image 18 of 60000
Processing image 19 of 60000
Processing image 20 of 60000
Processing image 21 of 60000
Processing image 22 of 60000
Processing image 23 of 60000
Processing image 24 of 60000
Processing image 25 of 60000
Processing image 26 of 60000
Processing image 27 of 60000
Processing image 28 of 60000
Processing image 29 of 60000
Processing image 30 of 60000
Processing image 31 of 60000
Processing image 32 of 60000
Processing image 33 of 60000
Processing image 34 of 60000
Processing image 35 of 

Processing image 343 of 60000
Processing image 344 of 60000
Processing image 345 of 60000
Processing image 346 of 60000
Processing image 347 of 60000
Processing image 348 of 60000
Processing image 349 of 60000
Processing image 350 of 60000
Processing image 351 of 60000
Processing image 352 of 60000
Processing image 353 of 60000
Processing image 354 of 60000
Processing image 355 of 60000
Processing image 356 of 60000
Processing image 357 of 60000
Processing image 358 of 60000
Processing image 359 of 60000
Processing image 360 of 60000
Processing image 361 of 60000
Processing image 362 of 60000
Processing image 363 of 60000
Processing image 364 of 60000
Processing image 365 of 60000
Processing image 366 of 60000
Processing image 367 of 60000
Processing image 368 of 60000
Processing image 369 of 60000
Processing image 370 of 60000
Processing image 371 of 60000
Processing image 372 of 60000
Processing image 373 of 60000
Processing image 374 of 60000
Processing image 375 of 60000
Processing

Processing image 656 of 60000
Processing image 657 of 60000
Processing image 658 of 60000
Processing image 659 of 60000
Processing image 660 of 60000
Processing image 661 of 60000
Processing image 662 of 60000
Processing image 663 of 60000
Processing image 664 of 60000
Processing image 665 of 60000
Processing image 666 of 60000
Processing image 667 of 60000
Processing image 668 of 60000
Processing image 669 of 60000
Processing image 670 of 60000
Processing image 671 of 60000
Processing image 672 of 60000
Processing image 673 of 60000
Processing image 674 of 60000
Processing image 675 of 60000
Processing image 676 of 60000
Processing image 677 of 60000
Processing image 678 of 60000
Processing image 679 of 60000
Processing image 680 of 60000
Processing image 681 of 60000
Processing image 682 of 60000
Processing image 683 of 60000
Processing image 684 of 60000
Processing image 685 of 60000
Processing image 686 of 60000
Processing image 687 of 60000
Processing image 688 of 60000
Processing

Processing image 971 of 60000
Processing image 972 of 60000
Processing image 973 of 60000
Processing image 974 of 60000
Processing image 975 of 60000
Processing image 976 of 60000
Processing image 977 of 60000
Processing image 978 of 60000
Processing image 979 of 60000
Processing image 980 of 60000
Processing image 981 of 60000
Processing image 982 of 60000
Processing image 983 of 60000
Processing image 984 of 60000
Processing image 985 of 60000
Processing image 986 of 60000
Processing image 987 of 60000
Processing image 988 of 60000
Processing image 989 of 60000
Processing image 990 of 60000
Processing image 991 of 60000
Processing image 992 of 60000
Processing image 993 of 60000
Processing image 994 of 60000
Processing image 995 of 60000
Processing image 996 of 60000
Processing image 997 of 60000
Processing image 998 of 60000
Processing image 999 of 60000
Processing image 1000 of 60000
Processing image 1001 of 60000
Processing image 1002 of 60000
Processing image 1003 of 60000
Proces

Processing image 1331 of 60000
Processing image 1332 of 60000
Processing image 1333 of 60000
Processing image 1334 of 60000
Processing image 1335 of 60000
Processing image 1336 of 60000
Processing image 1337 of 60000
Processing image 1338 of 60000
Processing image 1339 of 60000
Processing image 1340 of 60000
Processing image 1341 of 60000
Processing image 1342 of 60000
Processing image 1343 of 60000
Processing image 1344 of 60000
Processing image 1345 of 60000
Processing image 1346 of 60000
Processing image 1347 of 60000
Processing image 1348 of 60000
Processing image 1349 of 60000
Processing image 1350 of 60000
Processing image 1351 of 60000
Processing image 1352 of 60000
Processing image 1353 of 60000
Processing image 1354 of 60000
Processing image 1355 of 60000
Processing image 1356 of 60000
Processing image 1357 of 60000
Processing image 1358 of 60000
Processing image 1359 of 60000
Processing image 1360 of 60000
Processing image 1361 of 60000
Processing image 1362 of 60000
Processi

...

Processing image 57881 of 60000
Processing image 57882 of 60000
Processing image 57883 of 60000
Processing image 57884 of 60000
Processing image 57885 of 60000
Processing image 57886 of 60000
Processing image 57887 of 60000
Processing image 57888 of 60000
Processing image 57889 of 60000
Processing image 57890 of 60000
Processing image 57891 of 60000
Processing image 57892 of 60000
Processing image 57893 of 60000
Processing image 57894 of 60000
Processing image 57895 of 60000
Processing image 57896 of 60000
Processing image 57897 of 60000
Processing image 57898 of 60000
Processing image 57899 of 60000
Processing image 57900 of 60000
Processing image 57901 of 60000
Processing image 57902 of 60000
Processing image 57903 of 60000
Processing image 57904 of 60000
Processing image 57905 of 60000
Processing image 57906 of 60000
Processing image 57907 of 60000
Processing image 57908 of 60000
Processing image 57909 of 60000
Processing image 57910 of 60000
Processing image 57911 of 60000
Processi

Processing image 58381 of 60000
Processing image 58382 of 60000
Processing image 58383 of 60000
Processing image 58384 of 60000
Processing image 58385 of 60000
Processing image 58386 of 60000
Processing image 58387 of 60000
Processing image 58388 of 60000
Processing image 58389 of 60000
Processing image 58390 of 60000
Processing image 58391 of 60000
Processing image 58392 of 60000
Processing image 58393 of 60000
Processing image 58394 of 60000
Processing image 58395 of 60000
Processing image 58396 of 60000
Processing image 58397 of 60000
Processing image 58398 of 60000
Processing image 58399 of 60000
Processing image 58400 of 60000
Processing image 58401 of 60000
Processing image 58402 of 60000
Processing image 58403 of 60000
Processing image 58404 of 60000
Processing image 58405 of 60000
Processing image 58406 of 60000
Processing image 58407 of 60000
Processing image 58408 of 60000
Processing image 58409 of 60000
Processing image 58410 of 60000
Processing image 58411 of 60000
Processi

Processing image 58880 of 60000
Processing image 58881 of 60000
Processing image 58882 of 60000
Processing image 58883 of 60000
Processing image 58884 of 60000
Processing image 58885 of 60000
Processing image 58886 of 60000
Processing image 58887 of 60000
Processing image 58888 of 60000
Processing image 58889 of 60000
Processing image 58890 of 60000
Processing image 58891 of 60000
Processing image 58892 of 60000
Processing image 58893 of 60000
Processing image 58894 of 60000
Processing image 58895 of 60000
Processing image 58896 of 60000
Processing image 58897 of 60000
Processing image 58898 of 60000
Processing image 58899 of 60000
Processing image 58900 of 60000
Processing image 58901 of 60000
Processing image 58902 of 60000
Processing image 58903 of 60000
Processing image 58904 of 60000
Processing image 58905 of 60000
Processing image 58906 of 60000
Processing image 58907 of 60000
Processing image 58908 of 60000
Processing image 58909 of 60000
Processing image 58910 of 60000
Processi

Processing image 59380 of 60000
Processing image 59381 of 60000
Processing image 59382 of 60000
Processing image 59383 of 60000
Processing image 59384 of 60000
Processing image 59385 of 60000
Processing image 59386 of 60000
Processing image 59387 of 60000
Processing image 59388 of 60000
Processing image 59389 of 60000
Processing image 59390 of 60000
Processing image 59391 of 60000
Processing image 59392 of 60000
Processing image 59393 of 60000
Processing image 59394 of 60000
Processing image 59395 of 60000
Processing image 59396 of 60000
Processing image 59397 of 60000
Processing image 59398 of 60000
Processing image 59399 of 60000
Processing image 59400 of 60000
Processing image 59401 of 60000
Processing image 59402 of 60000
Processing image 59403 of 60000
Processing image 59404 of 60000
Processing image 59405 of 60000
Processing image 59406 of 60000
Processing image 59407 of 60000
Processing image 59408 of 60000
Processing image 59409 of 60000
Processing image 59410 of 60000
Processi

Processing image 59879 of 60000
Processing image 59880 of 60000
Processing image 59881 of 60000
Processing image 59882 of 60000
Processing image 59883 of 60000
Processing image 59884 of 60000
Processing image 59885 of 60000
Processing image 59886 of 60000
Processing image 59887 of 60000
Processing image 59888 of 60000
Processing image 59889 of 60000
Processing image 59890 of 60000
Processing image 59891 of 60000
Processing image 59892 of 60000
Processing image 59893 of 60000
Processing image 59894 of 60000
Processing image 59895 of 60000
Processing image 59896 of 60000
Processing image 59897 of 60000
Processing image 59898 of 60000
Processing image 59899 of 60000
Processing image 59900 of 60000
Processing image 59901 of 60000
Processing image 59902 of 60000
Processing image 59903 of 60000
Processing image 59904 of 60000
Processing image 59905 of 60000
Processing image 59906 of 60000
Processing image 59907 of 60000
Processing image 59908 of 60000
Processing image 59909 of 60000
Processi

In [9]:
save_data('./mnis_test.ctf', test_images, test_labels)

Processing image 1 of 10000
Processing image 2 of 10000
Processing image 3 of 10000
Processing image 4 of 10000
Processing image 5 of 10000
Processing image 6 of 10000
Processing image 7 of 10000
Processing image 8 of 10000
Processing image 9 of 10000
Processing image 10 of 10000
Processing image 11 of 10000
Processing image 12 of 10000
Processing image 13 of 10000
Processing image 14 of 10000
Processing image 15 of 10000
Processing image 16 of 10000
Processing image 17 of 10000
Processing image 18 of 10000
Processing image 19 of 10000
Processing image 20 of 10000
Processing image 21 of 10000
Processing image 22 of 10000
Processing image 23 of 10000
Processing image 24 of 10000
Processing image 25 of 10000
Processing image 26 of 10000
Processing image 27 of 10000
Processing image 28 of 10000
Processing image 29 of 10000
Processing image 30 of 10000
Processing image 31 of 10000
Processing image 32 of 10000
Processing image 33 of 10000
Processing image 34 of 10000
Processing image 35 of 

Processing image 338 of 10000
Processing image 339 of 10000
Processing image 340 of 10000
Processing image 341 of 10000
Processing image 342 of 10000
Processing image 343 of 10000
Processing image 344 of 10000
Processing image 345 of 10000
Processing image 346 of 10000
Processing image 347 of 10000
Processing image 348 of 10000
Processing image 349 of 10000
Processing image 350 of 10000
Processing image 351 of 10000
Processing image 352 of 10000
Processing image 353 of 10000
Processing image 354 of 10000
Processing image 355 of 10000
Processing image 356 of 10000
Processing image 357 of 10000
Processing image 358 of 10000
Processing image 359 of 10000
Processing image 360 of 10000
Processing image 361 of 10000
Processing image 362 of 10000
Processing image 363 of 10000
Processing image 364 of 10000
Processing image 365 of 10000
Processing image 366 of 10000
Processing image 367 of 10000
Processing image 368 of 10000
Processing image 369 of 10000
Processing image 370 of 10000
Processing

Processing image 758 of 10000
Processing image 759 of 10000
Processing image 760 of 10000
Processing image 761 of 10000
Processing image 762 of 10000
Processing image 763 of 10000
Processing image 764 of 10000
Processing image 765 of 10000
Processing image 766 of 10000
Processing image 767 of 10000
Processing image 768 of 10000
Processing image 769 of 10000
Processing image 770 of 10000
Processing image 771 of 10000
Processing image 772 of 10000
Processing image 773 of 10000
Processing image 774 of 10000
Processing image 775 of 10000
Processing image 776 of 10000
Processing image 777 of 10000
Processing image 778 of 10000
Processing image 779 of 10000
Processing image 780 of 10000
Processing image 781 of 10000
Processing image 782 of 10000
Processing image 783 of 10000
Processing image 784 of 10000
Processing image 785 of 10000
Processing image 786 of 10000
Processing image 787 of 10000
Processing image 788 of 10000
Processing image 789 of 10000
Processing image 790 of 10000
Processing

Processing image 1116 of 10000
Processing image 1117 of 10000
Processing image 1118 of 10000
Processing image 1119 of 10000
Processing image 1120 of 10000
Processing image 1121 of 10000
Processing image 1122 of 10000
Processing image 1123 of 10000
Processing image 1124 of 10000
Processing image 1125 of 10000
Processing image 1126 of 10000
Processing image 1127 of 10000
Processing image 1128 of 10000
Processing image 1129 of 10000
Processing image 1130 of 10000
Processing image 1131 of 10000
Processing image 1132 of 10000
Processing image 1133 of 10000
Processing image 1134 of 10000
Processing image 1135 of 10000
Processing image 1136 of 10000
Processing image 1137 of 10000
Processing image 1138 of 10000
Processing image 1139 of 10000
Processing image 1140 of 10000
Processing image 1141 of 10000
Processing image 1142 of 10000
Processing image 1143 of 10000
Processing image 1144 of 10000
Processing image 1145 of 10000
Processing image 1146 of 10000
Processing image 1147 of 10000
Processi

Processing image 1526 of 10000
Processing image 1527 of 10000
Processing image 1528 of 10000
Processing image 1529 of 10000
Processing image 1530 of 10000
Processing image 1531 of 10000
Processing image 1532 of 10000
Processing image 1533 of 10000
Processing image 1534 of 10000
Processing image 1535 of 10000
Processing image 1536 of 10000
Processing image 1537 of 10000
Processing image 1538 of 10000
Processing image 1539 of 10000
Processing image 1540 of 10000
Processing image 1541 of 10000
Processing image 1542 of 10000
Processing image 1543 of 10000
Processing image 1544 of 10000
Processing image 1545 of 10000
Processing image 1546 of 10000
Processing image 1547 of 10000
Processing image 1548 of 10000
Processing image 1549 of 10000
Processing image 1550 of 10000
Processing image 1551 of 10000
Processing image 1552 of 10000
Processing image 1553 of 10000
Processing image 1554 of 10000
Processing image 1555 of 10000
Processing image 1556 of 10000
Processing image 1557 of 10000
Processi

...

Processing image 7775 of 10000
Processing image 7776 of 10000
Processing image 7777 of 10000
Processing image 7778 of 10000
Processing image 7779 of 10000
Processing image 7780 of 10000
Processing image 7781 of 10000
Processing image 7782 of 10000
Processing image 7783 of 10000
Processing image 7784 of 10000
Processing image 7785 of 10000
Processing image 7786 of 10000
Processing image 7787 of 10000
Processing image 7788 of 10000
Processing image 7789 of 10000
Processing image 7790 of 10000
Processing image 7791 of 10000
Processing image 7792 of 10000
Processing image 7793 of 10000
Processing image 7794 of 10000
Processing image 7795 of 10000
Processing image 7796 of 10000
Processing image 7797 of 10000
Processing image 7798 of 10000
Processing image 7799 of 10000
Processing image 7800 of 10000
Processing image 7801 of 10000
Processing image 7802 of 10000
Processing image 7803 of 10000
Processing image 7804 of 10000
Processing image 7805 of 10000
Processing image 7806 of 10000
Processi

Processing image 8127 of 10000
Processing image 8128 of 10000
Processing image 8129 of 10000
Processing image 8130 of 10000
Processing image 8131 of 10000
Processing image 8132 of 10000
Processing image 8133 of 10000
Processing image 8134 of 10000
Processing image 8135 of 10000
Processing image 8136 of 10000
Processing image 8137 of 10000
Processing image 8138 of 10000
Processing image 8139 of 10000
Processing image 8140 of 10000
Processing image 8141 of 10000
Processing image 8142 of 10000
Processing image 8143 of 10000
Processing image 8144 of 10000
Processing image 8145 of 10000
Processing image 8146 of 10000
Processing image 8147 of 10000
Processing image 8148 of 10000
Processing image 8149 of 10000
Processing image 8150 of 10000
Processing image 8151 of 10000
Processing image 8152 of 10000
Processing image 8153 of 10000
Processing image 8154 of 10000
Processing image 8155 of 10000
Processing image 8156 of 10000
Processing image 8157 of 10000
Processing image 8158 of 10000
Processi

Processing image 8773 of 10000
Processing image 8774 of 10000
Processing image 8775 of 10000
Processing image 8776 of 10000
Processing image 8777 of 10000
Processing image 8778 of 10000
Processing image 8779 of 10000
Processing image 8780 of 10000
Processing image 8781 of 10000
Processing image 8782 of 10000
Processing image 8783 of 10000
Processing image 8784 of 10000
Processing image 8785 of 10000
Processing image 8786 of 10000
Processing image 8787 of 10000
Processing image 8788 of 10000
Processing image 8789 of 10000
Processing image 8790 of 10000
Processing image 8791 of 10000
Processing image 8792 of 10000
Processing image 8793 of 10000
Processing image 8794 of 10000
Processing image 8795 of 10000
Processing image 8796 of 10000
Processing image 8797 of 10000
Processing image 8798 of 10000
Processing image 8799 of 10000
Processing image 8800 of 10000
Processing image 8801 of 10000
Processing image 8802 of 10000
Processing image 8803 of 10000
Processing image 8804 of 10000
Processi

Processing image 9153 of 10000
Processing image 9154 of 10000
Processing image 9155 of 10000
Processing image 9156 of 10000
Processing image 9157 of 10000
Processing image 9158 of 10000
Processing image 9159 of 10000
Processing image 9160 of 10000
Processing image 9161 of 10000
Processing image 9162 of 10000
Processing image 9163 of 10000
Processing image 9164 of 10000
Processing image 9165 of 10000
Processing image 9166 of 10000
Processing image 9167 of 10000
Processing image 9168 of 10000
Processing image 9169 of 10000
Processing image 9170 of 10000
Processing image 9171 of 10000
Processing image 9172 of 10000
Processing image 9173 of 10000
Processing image 9174 of 10000
Processing image 9175 of 10000
Processing image 9176 of 10000
Processing image 9177 of 10000
Processing image 9178 of 10000
Processing image 9179 of 10000
Processing image 9180 of 10000
Processing image 9181 of 10000
Processing image 9182 of 10000
Processing image 9183 of 10000
Processing image 9184 of 10000
Processi

Processing image 9634 of 10000
Processing image 9635 of 10000
Processing image 9636 of 10000
Processing image 9637 of 10000
Processing image 9638 of 10000
Processing image 9639 of 10000
Processing image 9640 of 10000
Processing image 9641 of 10000
Processing image 9642 of 10000
Processing image 9643 of 10000
Processing image 9644 of 10000
Processing image 9645 of 10000
Processing image 9646 of 10000
Processing image 9647 of 10000
Processing image 9648 of 10000
Processing image 9649 of 10000
Processing image 9650 of 10000
Processing image 9651 of 10000
Processing image 9652 of 10000
Processing image 9653 of 10000
Processing image 9654 of 10000
Processing image 9655 of 10000
Processing image 9656 of 10000
Processing image 9657 of 10000
Processing image 9658 of 10000
Processing image 9659 of 10000
Processing image 9660 of 10000
Processing image 9661 of 10000
Processing image 9662 of 10000
Processing image 9663 of 10000
Processing image 9664 of 10000
Processing image 9665 of 10000
Processi